## Getting baseline values for G-Scores using all the data in using round-robin approach 

In [1]:
#importing the required libraries

import os, random
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix

random.seed(10)

C:\Users\USAKNAL\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
os.getcwd()
data_path = "/defects/src/data/Jureczko/collated_data/"
os.chdir(os.getcwd() + data_path)

In [3]:
projList = os.listdir()
projs = [p.split('_')[0] for p in projList]
print("List of projects:",projs)

List of projects: ['ant', 'camel', 'ivy', 'jedit', 'log4j', 'lucene', 'poi', 'velocity', 'xalan', 'xerces']


### Code to iteratively train a random forest classifier on a project and test it on all the other projects 

In [ ]:

tbl = [[0]*len(projs) for p in projs]

for i in range(len(projs)):
    print("\nIteration ",i)
    #fetching the training data
    trainData = pd.read_csv(projList[i])    
    X_train = trainData.loc[:,trainData.columns!='$<bug']
    y_train = trainData.loc[:,trainData.columns=='$<bug']
    for j in range(len(projs)):
        #making predictions on the other projects
        if (i != j):
            testData = pd.read_csv(projList[j])
            X_test = testData.loc[:, testData.columns!='$<bug']
            y_test = testData.loc[:, testData.columns=='$<bug']
            
            #training the classifier on current project
            clf = RandomForestClassifier(n_estimators=1000, n_jobs=1)
            clf.fit(X_train, y_train.values.ravel())
            y_pred = clf.predict(X_test)
            cm = confusion_matrix(y_test, y_pred)
            recall = cm[1][1]/(cm[1][1] + cm[1][0])
            pf = cm[0][1]/(cm[0][1] + cm[0][0])
            #computing the g-score
            g = 2/((1/recall) + (1/(1-pf)))
            
            print("Model trained on ",projs[i] + " " +  projs[j])
            tbl[i][j]=g

#printing the final g-score values for all the projects
print(tbl)


### Exporting the results to a csv file

In [ ]:
df = pd.DataFrame(tbl)
df.columns=projs
df.insert(0,'projects',projs)
print(df)
#print(os.getcwd())
df.to_csv("..//baseline_gscore.csv",index=False)